# WeRateDogs

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import requests
import os
import tweepy
import json
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Gathering Data

In [ ]:
#read twitter-archive-enhanced file
twitterarchive_df = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
#read first 5 rows 
twitterarchive_df.head()

In [ ]:
#download image predictions tweet
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)
#test response 

response

In [ ]:
#save downloaded file into my computer 

with open(os.path.join(url.split('/')[-1]),mode = "wb") as file:
        file.write(response.content)

#test if the file successfully saved 
os.listdir()

In [ ]:
#image predictions dataframe

imgpredictions_df = pd.read_csv('image-predictions.tsv', sep = "\t")

imgpredictions_df.head()

i don't have twitter developer account so i'll comment this part of code and use the project resources file

In [ ]:
#Accessing Project Data Without a Twitter Account
#import tweepy
#from tweepy import OAuthHandler
#import json
#from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
#consumer_key = 'HIDDEN'
#consumer_secret = 'HIDDEN'
#access_token = 'HIDDEN'
#access_secret = 'HIDDEN'

#auth = OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)

#api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
#tweet_ids = twitterarchive_df.tweet_id.values
#len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
#count = 0
#fails_dict = {}
#start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
#with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
#    for tweet_id in tweet_ids:
 #       count += 1
  #      print(str(count) + ": " + str(tweet_id))
   #     try:
    #        tweet = api.get_status(tweet_id, tweet_mode='extended')
     #       print("Success")
      #      json.dump(tweet._json, outfile)
       #     outfile.write('\n')
        #except tweepy.TweepError as e:
         #   print("Fail")
          #  fails_dict[tweet_id] = e
           # pass
#end = timer()
#print(end - start)
#print(fails_dict)

In [ ]:
with open('tweet-json copy', 'r') as f:
    getdata_df = pd.read_json(f, lines= True, encoding = 'utf-8')
    
# confirm
getdata_df.head()

In [ ]:


# select the columns of interest : 'id', 'favorite_count','retweet_count'
getdata_df = getdata_df.loc[:,['id','favorite_count','retweet_count']]

# check for the top 5 rows
getdata_df.head()



# Assessing Data

In [ ]:
#Assessing the twitter archive
twitterarchive_df

In [ ]:
#Assessing the image predictions
imgpredictions_df

In [ ]:
#Assessing the tweet json file data
getdata_df

# Getting the files information

In [ ]:
# Getting twitter archive information
twitterarchive_df.info()

In [ ]:
# Getting image predictions information
imgpredictions_df.info()

In [ ]:
# Getting tweet json information
getdata_df.info()

# Data Describe

In [ ]:
twitterarchive_df.describe()

In [ ]:
imgpredictions_df.describe()

In [ ]:
getdata_df.describe()

# Data Sample

In [ ]:
twitterarchive_df.sample(5)

In [ ]:
imgpredictions_df.sample(5)

In [ ]:
getdata_df.sample(5)

In [ ]:
# check for source column
twitterarchive_df.source.value_counts()

# Clean Data

In [ ]:
# make a copy of the datasets
twitterarchive_clean = twitterarchive_df.copy()
imgpredictions_clean = imgpredictions_df.copy()
getdata_clean =  getdata_df.copy()

# Quality

## 1.useless rows

### Define

Delete rows that have non-empty "retweeted_status_id and retweeted_status_user_id and retweeted_status_timestamp from twitterarchive_clean

### Code

In [ ]:
retweeted_index = twitterarchive_clean.query('retweeted_status_id != "NaN"').index
retweeted_index

In [ ]:
twitterarchive_clean.drop(retweeted_index, axis=0, inplace=True)
twitterarchive_clean.reset_index(drop=True, inplace=True)

### Test

In [ ]:
twitterarchive_clean.info()

## 2 useless columns

### Define 

Delete "in_reply_to_status_id and in_reply_to_user_id and retweeted_status_id and retweeted_status_user_id and  retweeted_status_timestamp" columns from twitterarchive_clean because we need the original tweets only

### Code

In [ ]:
twitterarchive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

### Test

In [ ]:
twitterarchive_clean.info()

## 3.expanded_urls Null value

### Define

Delete the null rows in expanded_urls

### Code

In [ ]:
twitterarchive_clean.dropna(subset=['tweet_id','expanded_urls'], inplace=True)
twitterarchive_clean.reset_index(drop=True, inplace=True)

In [ ]:
twitterarchive_clean.info()

##### now we don't have null in the files, Lets fix the data type!

#### twitterarchive_clean file

## 4.timestamp is object not datetime64

### Define

Change type of timestamp from string to datatime64

### Code

In [ ]:
twitterarchive_clean.timestamp = twitterarchive_clean.timestamp.astype('datetime64')

### Test

In [ ]:
twitterarchive_clean.info()

## 5. tweet_id is int64 not string

### Define

change type of tweet_id to object because we dont need to us it in math operation

### Code

In [ ]:
twitterarchive_clean.tweet_id = twitterarchive_clean.tweet_id.astype('str')

### Test

In [ ]:
twitterarchive_clean.info()

## 6.rating_numerator type is int64 not float

### Define

change the type of rating_numerator from int64 to float

### Code

In [ ]:
twitterarchive_clean.rating_numerator = twitterarchive_clean.rating_numerator.astype('float')

### Test

In [ ]:
twitterarchive_clean.info()

## 7.rating_denominator type is int64 not float

### Define 

change the type of rating_denominator from int64 to float

### Code

In [ ]:
twitterarchive_clean.rating_denominator = twitterarchive_clean.rating_denominator.astype('float')

### Test

In [ ]:
twitterarchive_clean.info()

#### imgpredictions_clean file

In [ ]:
imgpredictions_clean.info()

## 8.tweet_id is int64 not string

### Define 

change type of tweet_id from int64 to object

### code

In [ ]:
imgpredictions_clean.tweet_id = imgpredictions_clean.tweet_id.astype('str')

### Test

In [ ]:
imgpredictions_clean.info()

#### getdata_clean file

In [ ]:
getdata_clean.info()

## 9.id is int64 not string

### Define

change type of id from int64 to object

### Code

In [ ]:
getdata_clean.id = getdata_clean.id.astype('str')

### Test

In [ ]:
getdata_clean.info()

## decimals values

### Define

correct decimals values in rating numerator

### Code

In [ ]:
twitterarchive_clean[twitterarchive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

In [ ]:
tweets_correction_dict = {'883482846933004288': [13.50, 10],
                          '786709082849828864': [9.75, 10], 
                          '778027034220126208': [11.27, 10],
                          '680494726643068929': [11.26, 10]
                          }

for key, value in tweets_correction_dict.items():
    twitterarchive_clean.loc[twitterarchive_clean.query('tweet_id == "{}"'.format(key)).index,'rating_numerator'] = value[0]
    twitterarchive_clean.loc[twitterarchive_clean.query('tweet_id == "{}"'.format(key)).index,'rating_denominator'] = value[1]

### Test

In [ ]:
twitterarchive_clean[twitterarchive_clean.tweet_id.isin(list(tweets_correction_dict.keys()))][['tweet_id', 'rating_numerator', 'rating_denominator']]

In [ ]:
twitterarchive_clean[twitterarchive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

## 10.rating_numerator and rating_denominator should be in one column

### Define

creat rating column and merge rating_numerator and rating_denominator on it 

### Code

In [ ]:
twitterarchive_clean['rating'] = twitterarchive_clean.rating_numerator / twitterarchive_clean.rating_denominator

In [ ]:
twitterarchive_clean.drop(['rating_numerator', 'rating_denominator'], axis=1, inplace=True)

### Test

In [ ]:
twitterarchive_clean.head(1)

In [ ]:
twitterarchive_clean[twitterarchive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

## 11.wrong names "a, an, the, such".

### Define

If text has the word 'named', replace the name with the word after it. else, replace name with None.

### Code

In [ ]:
wrong_name = twitterarchive_clean.query('name in ["a", "an", "the", "such"]')
wrong_name

In [ ]:
def correct_name_func(text):
    split_list = text.split()
    if 'named' in split_list:
        name_index = split_list.index('named') + 1
        name = split_list[name_index]
        return name
    else:
        return None

In [ ]:
wrong_name['correct_name'] = wrong_name.apply(lambda x: correct_name_func(x['text']), axis=1)

In [ ]:
wrong_name

In [ ]:
correctname_dic = {}
for index, row in wrong_name.iterrows():
    correctname_dic[row.tweet_id] = row.correct_name
correctname_dic

In [ ]:
namecolumn_index = list(twitterarchive_clean.columns).index('name')
namecolumn_index

In [ ]:
namecolumn_index = list(twitterarchive_clean.columns).index('name')
for index, row in twitterarchive_clean.iterrows():
    if row['tweet_id'] in correctname_dic.keys():
        twitterarchive_clean.iloc[index, namecolumn_index] = correctname_dic[row.tweet_id]

### Test

In [ ]:
twitterarchive_clean.query('name in ["a", "an", "the", "such"]')

# Tidiness

## twitterarchive_clean file has 1 variable in 4 columns
"doggo, floofer, pupper, puppo"

### Define

creat dog_stage column by joining The values of four columns "doggo,floofer,pupper,puppo"

### Code

In [ ]:
alldogs_type = ['doggo', 'floofer', 'pupper', 'puppo']

twitterarchive_clean[alldogs_type] = twitterarchive_clean[alldogs_type].replace('None', np.nan)

twitterarchive_clean['dog_stage'] = twitterarchive_clean[alldogs_type].\
                                        apply(lambda x: ', '.join(x.dropna().astype(str)),axis =1)
twitterarchive_clean.dog_stage = twitterarchive_clean.dog_stage.replace('', np.nan).astype('category')

# delete the 4 columns
twitterarchive_clean = twitterarchive_clean.drop(columns = alldogs_type, axis =1)

In [ ]:
# check for the values of the new column
twitterarchive_clean.dog_stage.value_counts()

### Test

In [ ]:
# check for the values of the new column
twitterarchive_clean

## imgpredictions_clean file columns p1,p1_dog_p1_conf,...etc to dog_breed, confidence

### Define 

rename the No breed values with np.nan and replace the underscore with space and title all breeds values

### Code

In [ ]:
breed = []
confidence = []


def dog_breed_confidence(data):
    if data.p1_dog:
        breed.append(data.p1)
        confidence.append(data.p1_conf)
    elif data.p2_dog:
        breed.append(data.p2)
        confidence.append(data.p2_conf)
    elif data.p3_dog :
        breed.append(data.p3)
        confidence.append(data.p3_conf)
    else:
        breed.append('No breed')
        confidence.append(0)
      
imgpredictions_clean.apply(dog_breed_confidence,axis =1)

imgpredictions_clean['dog_breed'] = breed
imgpredictions_clean['confidence'] = confidence

imgpredictions_clean.drop(columns = ['p1', 'p1_dog', 'p1_conf' , 'p2', 'p2_dog',
                                 'p2_conf' , 'p3', 'p3_dog', 'p3_conf'],axis=1, inplace =True)

imgpredictions_clean.replace('No breed',np.nan, inplace=True)

imgpredictions_clean.dog_breed= imgpredictions_clean.dog_breed.str.replace('_',' ').str.title()

### Test

In [ ]:
imgpredictions_clean.head()

## merge twitterarchive_clean and getdata_clean

### Define

use the merge function to merge twitterarchive_clean and getdata_clean on tweet_id column (left join)


use the merge function to merge `twitterarchive_clean` and `imgpredictions_clean` on tweet_id column (inner join)

### Code

In [ ]:
# rename the id column in getdata_clean to tweet_id
getdata_clean.columns = ['tweet_id', 'favorite_count', 'retweet_count']

### Test

In [ ]:
getdata_clean

In [ ]:
# merge twitterarchive_clean and getdata_clean on tweet_id column (left join)
twitterarchive_clean = pd.merge(twitterarchive_clean, getdata_clean , how = 'left' , on = 'tweet_id')

# use the merge function to merge `twitterarchive_clean` and `imgpredictions_clean` on tweet_id column (inner join) 
# and make master dataset
master_dataset = pd.merge(twitterarchive_clean, imgpredictions_clean , how = 'inner' , on = 'tweet_id')

### Test

In [ ]:
master_dataset

In [ ]:
master_dataset.info()

## new columns for day_name and month for more

### Define

creat month column

creat day name column

### code

In [ ]:
# extract the month name
master_dataset['month'] = master_dataset.timestamp.apply(lambda x: x.month_name())
#extarct the day_name
master_dataset['day_name'] = master_dataset.timestamp.apply(lambda x: x.day_name())

### Test

In [ ]:
# check for the top 5 rows in columns timestamp, day_name and month
master_dataset.loc[:5,['timestamp','day_name','month']]

In [ ]:
master_dataset

# Storing Data

In [ ]:
# Store the data after combinig and cleaning 
master_dataset.to_csv('twitter_archive_master.csv',encoding='utf-8',index=False)

# Visualization & Analysis

### most source used to share dog's photo

In [ ]:
plt.title("Distribution of Tweets'Source")
master_dataset.source.value_counts().sort_values().plot(kind ='barh')
plt.xlabel('Total Tweets')
plt.ylabel('Source');

In [ ]:
# percentage of sources
master_dataset.source.value_counts() / master_dataset.source.value_counts().sum()

#### Twitter app in Iphone has the most share 98%

## the most common dog stage

In [ ]:
# select the dog_ stage frequencies
master_dataset.dog_stage.value_counts().plot('bar')
plt.title("Distribution of Dog Stages")
plt.xlabel('Dog Stage')
plt.ylabel('Frequency');

#### Pupper is the most common dog stage in our dataset

## the most popular day to post a dog photo

In [ ]:
master_dataset.day_name.value_counts().plot('bar')
plt.title("Distribution of Tweets over Days")
plt.xlabel('Week Days')
plt.ylabel('Frequency');

#### monday the most popular day to post a dog photo